In [2]:
from transformers import RobertaTokenizer, RobertaModel

# Load the ChemBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
model = RobertaModel.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')



# Example SMILES string
smiles = 'CCO'

# Tokenize the SMILES string
inputs = tokenizer(smiles, return_tensors='pt')

# Get the model outputs
outputs = model(**inputs)

# The output is a tuple with the last hidden state as the first element
last_hidden_state = outputs[0]


tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/9.43k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/3.21k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/179M [00:00<?, ?B/s]

In [3]:
print(last_hidden_state.shape)

torch.Size([1, 3, 768])


In [4]:
last_hidden_state

tensor([[[ 0.8511,  2.1786, -0.7335,  ..., -0.2723, -1.0461,  0.6268],
         [ 1.3393,  1.9692, -0.6810,  ...,  0.0738,  0.0690, -0.2010],
         [ 1.2621,  2.0117, -0.7737,  ..., -0.2274, -1.0789,  0.6698]]],
       grad_fn=<NativeLayerNormBackward0>)

In [5]:
mean1 = last_hidden_state.mean(dim=1)

In [7]:
mean1.shape

torch.Size([1, 768])

In [8]:
from transformers import RobertaTokenizer, RobertaModel

# Download the tokenizer and model and save them locally
tokenizer = RobertaTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
model = RobertaModel.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')

# Save the tokenizer and model locally
tokenizer.save_pretrained('./ChemBERTa-tokenizer')
model.save_pretrained('./ChemBERTa-model')


In [9]:
tokenizer = RobertaTokenizer.from_pretrained('./ChemBERTa-tokenizer')
model = RobertaModel.from_pretrained('./ChemBERTa-model')

# Example SMILES string
smiles = 'CCO'

# Tokenize the SMILES string
inputs = tokenizer(smiles, return_tensors='pt')

# Get the model outputs
outputs = model(**inputs)

# The output is a tuple with the last hidden state as the first element
last_hidden_state = outputs[0]


In [10]:
last_hidden_state.shape

torch.Size([1, 3, 768])

In [16]:
import torch
from transformers import RobertaTokenizer, RobertaModel
import pubchempy as pcp

# Load the ChemBERTa tokenizer and model from local files
tokenizer = RobertaTokenizer.from_pretrained('./ChemBERTa-tokenizer')
model = RobertaModel.from_pretrained('./ChemBERTa-model')

def name_to_smiles_pubchem(name):
    """Convert a chemical name to a SMILES string using PubChemPy."""
    compounds = pcp.get_compounds(name, 'name')
    if compounds:
        return compounds[0].canonical_smiles
    else:
        return None

def smiles_to_embedding(smiles):
    """Convert a SMILES string to a ChemBERTa embedding."""
    inputs = tokenizer(smiles, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    # The output is a tuple with the last hidden state as the first element
    last_hidden_state = outputs[0]
    # We typically use the embedding of the [CLS] token (first token)
    embedding = last_hidden_state[:, 0, :]
    return embedding

def substrate_to_embedding(name):
    """Convert a substrate name to a ChemBERTa embedding."""
    smiles = name_to_smiles_pubchem(name)
    if smiles:
        embedding = smiles_to_embedding(smiles)
        return embedding
    else:
        raise ValueError(f"SMILES not found for substrate: {name}")

# Example usage
substrate_name = "reduced acetylpyridine adenine dinucleotide"
embedding = substrate_to_embedding(substrate_name)

print(f"Embedding for {substrate_name}:")
print(embedding)


ValueError: SMILES not found for substrate: reduced acetylpyridine adenine dinucleotide